### LangSmith Walkthrough

#### Resources
- [langsmith cookbook](https://github.com/langchain-ai/langsmith-cookbook/blob/main/tracing-examples/traceable/tracing_without_langchain.ipynb)

In [ ]:
!pip install langsmith langchain anthropic

In [ ]:
import langsmith, os, sys
import anthropic as anthropic_base
from langsmith import traceable
from pathlib import Path 

_root = "/home/zjc1002/Mounts/code/admin/"


sys.path.append(_root)
from api_keys import _api_keys


### Langsmith API Parameters

In [ ]:
!export LANGSMITH_TRACING_v2=true

In [ ]:
LANGSMITH_TRACING=True
LANGSMITH_ENDPOINT='https://api.smith.langchain.com'
LANGSMITH_PROJECT="LangSmith Basics"

LANGSMITH_API_KEY= _api_keys["LANGSMITH_API_KEY"]
ANTHROPIC_API_KEY=_api_keys['ANTHROPIC_API_KEY']
ANTHROPIC_MODEL='claude-3-haiku-20240307'

os.environ["LANGCHAIN_ENDPOINT"] = LANGSMITH_ENDPOINT
os.environ["LANGCHAIN_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGSMITH_PROJECT

!export LANGSMITH_TRACING=true

#### Create Sample Parameters to test LangSmith With 

In [ ]:
system_p = "You are a helpful assistant. Please respond to the user's request only based on the given context."
question = 'where are the best grits in Charlotte'
context = "Charlotte has alot of grits. But Hands down the best grits are from The Flying Biscuit, with a close second being Waffle House. "

### Create LLM INSTANCE
##### Note: I am using anthropic, but u should be able to use any model open source or not

In [ ]:
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
owner = client.__class__.__module__.split('.')[0]

#### Trace a basic function

- **@traceable** decorator is ment to manage the logging of events from the client using a RunTree, which tracks your application. Each RunTree is required to have the below paramters
   - **name:str**: used to id the components purpose
   - **run_type:str**: 'llm','chain', or 'tool'. describes the type of interaction / activity being logged
   - **inputs:dict**: the inputs to the component
   - **outputs:Optional[dict]**: the (optinal) returned values from the component
   - **error: Optional[str]**: any error messages that may have arisin during the call


In [ ]:
# Use the @traceable decorator with the 'project_name' parameter to log traces to LangSmith
# Ensure that the LANGCHAIN_TRACING_V2 environment variables are set for @traceable to work

@traceable(run_type="llm"
, name=LANGSMITH_PROJECT
, metadata = {'ls_provider':owner ,'ls_model_name':ANTHROPIC_MODEL}
)

def argument_generator(client, user_input: str, model = None , additional_system_prompt_details:str = ""):

    assert model is not None, "Model must be specified, only ANTHROPIC models currently supported"

    result = client.messages.create(
        model= model
        , max_tokens = 1000
        , temperature = .1
        , system = f"you are a general use chatbot tying to help people awnser questions. {additional_system_prompt_details}. The current time is {datetime.now()}"
        , messages=[
            {"role": "user", "content": user_input}]
    )

    return result.content[0]


@traceable(name=LANGSMITH_PROJECT)
def argument_chain(client,  model, user_input: str, additional_system_prompt_details:str = ""):

    argument = argument_generator(client
                                  , user_input
                                  , model=model
                                  , additional_system_prompt_details=additional_system_prompt_details
                                  )

    return argument

argument_chain(client, ANTHROPIC_MODEL, question)

### Tracing LLM Calls Explicitly with RunTree API

###### *enables you to manually create runs and children runs to assemble your trace*

In [ ]:
from langsmith.run_trees import RunTree
from uuid import uuid4

#specify run_id for tracking (optional)
run_id = uuid4()

#create a top-level run
pipeline = RunTree(name=LANGSMITH_PROJECT
                   , run_type="chain"
                   , inputs = {"question":question}
                   , metadata={'ls_provider':owner, 'ls_model_name':ANTHROPIC_MODEL}
                   , id= run_id
                   )

#post the tree to the LangSmith platform for storage and later analysis
pipeline.post()

#ask a question given specific context
messages = [
  { "role": "user", "content": f"Question: {question}\nContext: {context}"}
]


#create a child run (passing query and context to llm)
child_llm_run = pipeline.create_child(
    name = 'Anthropic Call'
    , run_type = 'llm'
    , inputs= messages
)

child_llm_run.post()

#generate simple completion
result = client.messages.create(
      model= ANTHROPIC_MODEL
      , max_tokens = 1000
      , temperature = .1
      , system = system_p
      , messages=messages
    )


#End the child trace and log it
child_llm_run.end(outputs=result)
child_llm_run.patch()


#End the parent trace and log it
pipeline.end(outputs={'anwser':result.content[0]})
pipeline.patch()

In [ ]:
#USE LANGSMITH CLIENT TO GET URL OF CURRNET RUNID BEING TRACKED
from langsmith import Client
_client = Client()
run = _client.read_run(run_id)
print(run.url)

#DELETE TRACES TIED TO PROJECT
Client().delete_project(project_name=LANGSMITH_PROJECT)